In [1]:
from casatasks import tclean, split, listobs, plotants, imstat
from casatasks import uvcontsub, flagdata, gencal, plotweather, setjy, gaincal, bandpass, applycal, fluxscale, importgmrt
try:
    from casaplotms import plotms
except:
    import os
    os.system('pip install casaplotms')
    from casaplotms import plotms
from pathlib import Path
from casatools import image as IA
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import numpy as np
import io
import base64
from os import path, makedirs, remove, getcwd
import glob
from shutil import rmtree
from matplotlib import rcParams, rc
import os
# helpers

def __clear_tmp(wd,cimagename):
    filelist = glob.glob(path.join(wd, cimagename+'.*'))
    for f in filelist:
        try:
            rmtree(f)
        except OSError:
            remove(f)
def save_fig(plt, fig, kind='base64', output='output.jpg'):
    
    if kind == 'base64':
        buf = io.BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight',
                    transparent=True, pad_inches=0)
        buf.seek(0)
        string = base64.b64encode(buf.read())
        plt.close()
        return string
    elif kind == 'plot':
        plt.show()
        return 'plotted'
    else :
        if not path.exists('output'):
            makedirs('output')
        newPath = 'output/'+output
        opt = newPath
        if path.exists(newPath):
            numb = 1
            while path.exists(newPath):
                newPath = "{0}_{2}{1}".format(
                    *path.splitext(opt) + (numb,))
                try :
                    if path.exists(newPath):
                        numb += 1 
                except:
                    pass               
        fig.savefig(newPath, format=kind, bbox_inches='tight',
                    pad_inches=0)
        print("saved {}".format(newPath))
        plt.close()
        return newPath

cwd = getcwd()
#dsource = '/home/avi/CASA/tests/test_d/atcow2018/DDTC022_28AUG2018.LTA_RRLL.RRLLFITS.fits' # ms file as visibility input


In [2]:
calcsource = '/home/avi/CASA/uGMRT/tst2505/TEST2505_B0329_100MHZ_GWB_2.MS'
csource = '/home/avi/CASA/uGMRT/cal_d/0329_100MHz_GWB_spectralimaging_LL.capture.selfcal.ms'
# split(vis=calcsource,outputvis=csource,correlation='LL') # ,datacolumn='corrected') #default

## Calibrating for spectral imaging
- ## pmcalf
    - field 0 : 3C48
    - close to the target
    - same atmospheric effect
- ## bpcalf
    - field 2 : 3C48
    - strong
    - long duration
    - to provide adequate per channel sensitivity
    


In [26]:
pmcalf = '0'
pmcals = '1'
bpcalf = '0'
bpcals = '3'
fdcalf = '0'
fdcals = '1'
targets = '1'
targetf = '1'

## Proceure
    - Solve for G on the bpcalf
    - Solve for B on the bpcalf using G
    - Solve for G on the pmcalf and fdcalf using B obtained
    - SCALE G for pbcalf according to fdcalf solution
    - APPLY B and G solution to the target
    - Direction-Dependent calibration
    - PB calibration and cross-calibration (or phase-referencing; for phase only)
    - Image
    
 > - **G** : Electronic Gain response due to components in the signal path between the feed and the correlator. The complex gain term includes the scale factor for absolute flux density calibration and may include **phase and amplitude corrections** due to change in the atmosphere. These gains are **polarization-dependent**
 > - **B** : Bandpass (frequency-dependent) response, such as that introduced by spectral filters in the electronic transmission system
     


In [8]:
# original author : Ruta Kale, Ishwara Chandra
#

# converted for SETI pipeline by @avialxee
# TODO : write a dictionary


from casatasks import tclean, split, listobs, plotants, clearcal, visstat
from casatasks import uvcontsub, flagdata, gencal, plotweather, setjy, gaincal, bandpass, applycal, fluxscale, importgmrt
try:
    from casaplotms import plotms
except:
    import os
    os.system('pip install casaplotms')
    from casaplotms import plotms
from pathlib import Path
from casatools import image as IA
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import numpy as np
import io
import base64
from os import path, makedirs, remove, getcwd
import glob
from shutil import rmtree
from matplotlib import rcParams, rc

from casatools import ms, msmetadata
import os
from casatasks import casalog, flagdata
import numpy as np
casalog.filter('DEBUG1')
msmd = msmetadata()
from capturelib import *

msmd=msmetadata()


In [11]:
listobs(csource, listfile='0329_100MHz_GWB_spectralimaging.txt', overwrite=True)
with open('0329_100MHz_GWB_spectralimaging.txt') as f:
    print(f.read())

           MeasurementSet Name:  /home/avi/CASA/uGMRT/cal_d/0329_100MHz_GWB_spectralimaging_LL.capture.selfcal.ms      MS Version 2
   Observer: PASV TEST     Project:   
Observation: GMRT
Data records: 107880       Total elapsed time = 1696.51 seconds
   Observed from   26-Jan-2022/14:59:10.6   to   26-Jan-2022/15:27:27.1 (TAI)

   ObservationID = 0         ArrayID = 0
  Date        Timerange (TAI)          Scan  FldId FieldName             nRows     SpwIds   Average Interval(s)    ScanIntent
  26-Jan-2022/14:59:10.6 - 15:00:52.6     1      0 3C48                      8265  [0]  [5.37] 
              15:03:55.1 - 15:14:12.5     2      1 B0329+54                 50025  [0]  [5.37] 
              15:17:15.0 - 15:27:27.1     3      0 3C48                     49590  [0]  [5.37] 
           (nRows = Total number of rows per scan) 
Fields: 2
  ID   Code Name                RA               Decl           Epoch   SrcId      nRows
  0         3C48                01:37:41.303328 +33.09.35.1442

In [29]:
# gainspw, mygoodchans, flagspw, mypol = getgainspw(csource)
# gainspw, mygoodchans, flagspw, mypol
mygoodchans = '0:500~600'   # used for visstat
flagspw = '0:101~1900'
gainspw = '0:201~1800'
gainspw2 = ''   # central good channels after split file for self-cal
myampcals=pmcalf
mypcals = bpcalf
mytargets = targetf

## pre cal

In [34]:
# flagdata(vis=csource,
#         mode='unflag', 
#          field='',
#          spw='',
#          antenna='', 
#          timerange=''
#         )


print(f'Clip at high amp levels')
flagdata(vis=csource,mode="clip", spw=flagspw, field=str(','.join(myampcals)), clipminmax=[0.0,80.0], datacolumn="DATA",
         clipoutside=True, clipzeros=True, extendpols=False,
         action="apply",flagbackup=True, savepars=False, overwrite=True, writeflags=True)
flagdata(vis=csource,mode="clip", spw=flagspw, field=str(','.join(mypcals)), clipminmax=[0.0,60.0], datacolumn="DATA",clipoutside=True, clipzeros=True, extendpols=False,
         action="apply",flagbackup=True, savepars=False, overwrite=True, writeflags=True)
# After clip, now flag using 'tfcrop' option for flux and phase cal tight flagging
print(f'After clip, now flag using "tfcrop" option for flux and phase cal tight flagging')
flagdata(vis=csource,mode="tfcrop", datacolumn="DATA", field=str(','.join(mypcals)), ntime="scan",
         timecutoff=5.0, freqcutoff=5.0, timefit="line",freqfit="line",flagdimension="freqtime", 
         extendflags=False, timedevscale=5.0,freqdevscale=5.0, extendpols=False,growaround=False,
         action="apply", flagbackup=True,overwrite=True, writeflags=True)
# 
print(f'Now extend the flags (80% more means full flag, change if required)')
flagdata(vis=csource, mode="extend", spw=flagspw, field=str(','.join(mypcals)), datacolumn="DATA", clipzeros=True,
         ntime="scan", extendflags=False, extendpols=True,growtime=80.0, growfreq=80.0,growaround=False,
         flagneartime=False, flagnearfreq=False, action="apply", flagbackup=True,overwrite=True, writeflags=True)
#==================######### target flagging ### clip first
#        if target == True:
print(f'######### target flagging ### clip first')
if mytargets !=[]:
        flagdata(vis=csource, mode="clip", spw=flagspw, field=str(','.join(mytargets)), clipminmax=[0.0,30.0],
                 datacolumn="DATA",clipoutside=True, clipzeros=True, extendpols=False, 
                 action="apply",flagbackup=True, savepars=False, overwrite=True, writeflags=True)
        print(f'flagging with tfcrop before calibration')
#                        default(flagdata)
        flagdata(vis=csource,mode="tfcrop", datacolumn="DATA", field=str(','.join(mytargets)), ntime="scan",
             timecutoff=6.0, freqcutoff=6.0, timefit="poly",freqfit="poly",flagdimension="freqtime",
                  extendflags=False, timedevscale=5.0,freqdevscale=5.0, extendpols=False,growaround=False,
                  action="apply", flagbackup=True,overwrite=True, writeflags=True)
        print(f'Now extend the flags (80% more means full flag, change if required) - 90%')
        flagdata(vis=csource,mode="extend",spw=flagspw,field=str(','.join(mytargets)),datacolumn="DATA",clipzeros=True,
              ntime="scan", extendflags=False, extendpols=True,growtime=90.0, growfreq=90.0,growaround=False,
                  flagneartime=False, flagnearfreq=False, action="apply", flagbackup=True,overwrite=True, writeflags=True)
        print(f'Now summary')
        flagdata(vis=csource,mode="summary",datacolumn="DATA", extendflags=True, 
                  name=csource+'summary.split', action="apply", flagbackup=True,overwrite=True, writeflags=True)        
print(f"A flagging summary is provided for the MS file.")
flagsummary(csource)

Clip at high amp levels
After clip, now flag using "tfcrop" option for flux and phase cal tight flagging
Now extend the flags (80% more means full flag, change if required)
######### target flagging ### clip first
flagging with tfcrop before calibration
Now extend the flags (80% more means full flag, change if required) - 90%
Now summary
A flagging summary is provided for the MS file.
Flagging percentage:
antenna C00 0.37431727527113456
antenna C01 0.3988535438681869
antenna C02 0.42806759029129593
antenna C03 0.3138116787750278
antenna C04 0.38587254067018906
antenna C05 0.5197221630805061
antenna C06 0.9561065571989711
antenna C08 0.7379211493673525
antenna C09 0.6204810901001112
antenna C10 0.483481266076891
antenna C11 0.46772346878476084
antenna C12 0.49215789896760287
antenna C13 0.3772638000556174
antenna C14 0.45007147698832034
antenna E02 0.6108675437986653
antenna E03 0.3761164205888487
antenna E04 0.16028008551167966
antenna E05 0.3007627833008899
antenna E06 0.3225290470314

In [35]:
# time v elevation
# time v amplitude
plotms(vis=csource,
       #xaxis='time',yaxis='elevation',correlation='RR,LL',
       xaxis='time',yaxis='amp',correlation='', field='',
       #avgchannel='2008',
       spw='0', coloraxis='field', #timerange='2018/08/28/12:59:04.3564~2018/08/28/17:50:03.3819'
        )

# Inspect Note: try with and without time range

## G on bpcalf

In [42]:
def mygaincal_ap2(myfile,mycal,myref,myflagspw,myuvracal,calsuffix):
        gtable = [str(myfile)+'.K1'+calsuffix, str(myfile)+'.B1'+calsuffix ]
        gaincal(vis=myfile, 
                caltable=str(myfile)+'.AP.G'+calsuffix, # output
                spw =myflagspw,
                uvrange=myuvracal,
                #append=True,
                #field=mycal,
                selectdata=True, scan=mycal,
                solint = '120s',
                refant = myref, 
                minsnr = 2.0, solmode ='L1R', gaintype = 'G', calmode = 'ap', rmsthresh=2.0,
                gaintable = gtable, interp = ['nearest,nearestflag', 'nearest,nearestflag' ], parang = True )
        return gtable

def mygaincal_ap23(myfile,mycal,myref,myflagspw,myuvracal,calsuffix):
        gtable = [str(myfile)+'.K1'+calsuffix, str(myfile)+'.B1'+calsuffix ]
        gaincal(vis=myfile, 
                caltable=str(myfile)+'.AP.G'+calsuffix, 
                spw =myflagspw,
                uvrange=myuvracal,
                append=True,
                field=mycal,
                solint = '120s',
                refant = myref, 
                minsnr = 2.0, solmode ='L1R', gaintype = 'G', calmode = 'ap', rmsthresh=2.0,
                gaintable = gtable, interp = ['nearest,nearestflag', 'nearest,nearestflag' ], parang = True )
        return gtable


gainspw

'0:201~1800'

In [84]:
ref_ant = 'C00'
clearcal(vis=csource)
uvracal=''
for i in range(len(myampcals)):
    setjy(vis=csource, spw=flagspw, field=bpcalf)#myampcals[i])

print('Delay calibration  using the first flux calibrator in the list - should depend on which is less flagged')
mycalsuffix = '_selfcal'
gntable=str(csource)+'.K1'+mycalsuffix
if os.path.isdir(gntable) == True:
    os.system('rm -rf '+gntable)
gaincal(vis=csource, caltable=gntable, spw =flagspw, selectdata=True, scan=bpcals,
        solint='300s', refant=ref_ant, solnorm= True, gaintype='K', gaintable=[], parang=True)


2022-02-11 08:32:15	WARN	calibrater::setvi(bool,bool)	Forcing use of OLD VisibilityIterator.


Delay calibration  using the first flux calibrator in the list - should depend on which is less flagged


In [85]:
# kcorrfield=myampcals[0]
print('an initial bandpass')
gntable_AP_G0=csource+'.AP.G0'+mycalsuffix
if os.path.isdir(gntable_AP_G0) == True:
    print(os.system('rm -rf '+gntable_AP_G0))
    
print(gntable_AP_G0)
gaincal(vis=csource, caltable=gntable_AP_G0, 
        #field=str(','.join(mybpcals)), 
        selectdata=True, scan=bpcals,
        spw =flagspw,
        solint = 'int', refant = ref_ant, minsnr = 2.0, solmode = 'L1R', gaintype = 'G', calmode = 'ap', 
        gaintable = [str(csource)+'.K1'+mycalsuffix], 
        interp = ['nearest,nearestflag', 'nearest,nearestflag' ], parang = True
       )


an initial bandpass
0
/home/avi/CASA/uGMRT/cal_d/0329_100MHz_GWB_spectralimaging_LL.capture.selfcal.ms.AP.G0_selfcal
7 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:03:57.8
7 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:04:03.2
6 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:04:08.5
7 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:04:13.9
7 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:04:19.3
7 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:04:24.6
6 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:04:30.0
7 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:04:35.4
7 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:04:40.7
7 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:04:46.1
6 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:04:51.5
7 of 30 solutions flagged due to SNR < 2 in spw=0 at 2022/

In [86]:
if os.path.isdir(str(csource)+'.B1'+mycalsuffix) == True:
    os.system('rm -rf '+str(csource)+'.B1'+mycalsuffix)
bptable=str(csource)+'.B1'+mycalsuffix
print('bandpass')
bandpass(vis=csource, caltable=bptable, 
         spw =flagspw, 
         #field=str(','.join(mybpcals)), 
         selectdata=True, scan=bpcals,
         solint='inf', refant=ref_ant, solnorm = True,
    minsnr=2.0, fillgaps=8, parang = True, gaintable=[str(csource)+'.K1'+mycalsuffix,gntable_AP_G0], 
         interp=['nearest,nearestflag','nearest,nearestflag'])
print('do a gaincal on all calibrators')
# mycals=myampcals+mypcals


bandpass
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1799) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1798) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1797) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1796) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1795) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1794) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1793) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1792) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1791) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1790) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to SNR < 2 in spw=0 (chan=1789) at 2022/01/26/15:08:57.9
2 of 25 solutions flagged due to S

In [44]:
mycals=[pmcals]


In [102]:
i=0
if os.path.isdir(str(csource)+'.AP.G'+mycalsuffix) == True:
    os.system('rm -rf '+str(csource)+'.AP.G'+mycalsuffix)
# for i in range(len(mycals)):
#     print(f'gaincal for {mycals[i]}')
#     if i==0:
#         mygaincal_ap2(csource,mycals[0],ref_ant,gainspw,uvracal,mycalsuffix)
#     else:
#         mygaincal_ap23(csource,mycals[i],ref_ant,gainspw,uvracal,mycalsuffix)
myfile=csource
calsuffix = mycalsuffix

gtable = [str(myfile)+'.K1'+calsuffix, str(myfile)+'.B1'+calsuffix ]
gaincal(vis=myfile, 
        caltable=str(myfile)+'.AP.G'+calsuffix, 
        spw =flagspw,
        uvrange=uvracal,
        #append=True,
        field='0',
        #selectdata=True, scan='0',
        solint = '300s',
        refant = ref_ant, 
        minsnr = 2.0, solmode ='L1R', gaintype = 'G', calmode = 'ap', rmsthresh=2.0,
        gaintable = gtable, interp = ['nearest,nearestflag', 'nearest,nearestflag' ], parang = True )


5 of 24 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:27:21.7


In [103]:
gaincal(vis=myfile, 
        caltable=str(myfile)+'.AP.G'+calsuffix, 
        spw =flagspw,
        uvrange=uvracal,
        append=True,
        field='0',
        #selectdata=True, scan='0',
        solint = '300s',
        refant = ref_ant, 
        minsnr = 2.0, solmode ='L1R', gaintype = 'G', calmode = 'ap', rmsthresh=2.0,
        gaintable = gtable, interp = ['nearest,nearestflag', 'nearest,nearestflag' ], parang = True )


5 of 24 solutions flagged due to SNR < 2 in spw=0 at 2022/01/26/15:27:21.7


In [109]:
myampcals=['0','1']
print('Get flux scale')
if os.path.isdir(str(csource)+'.fluxscale'+mycalsuffix) == True:
    os.system('rm -rf '+str(csource)+'.fluxscale'+mycalsuffix)
#############################
if mypcals !=[]:
    if '3C286' in myampcals:
        myfluxscale= getfluxcal2(csource,'3C286',str(', '.join(mypcals)),mycalsuffix)
        myfluxscaleref = '3C286'
    elif '3C147' in myampcals:
        myfluxscale= getfluxcal2(csource,'3C147',str(', '.join(mypcals)),mycalsuffix)
        myfluxscaleref = '3C147'
    else:
        myfluxscale= getfluxcal2(csource,myampcals[0],str(', '.join(mypcals)),mycalsuffix)
        myfluxscaleref = myampcals[0]
    print(myfluxscale)
    mygaintables =[str(csource)+'.fluxscale'+mycalsuffix,str(csource)+'.K1'+mycalsuffix, str(csource)+'.B1'+mycalsuffix]
else:
    mygaintables =[str(csource)+'.AP.G'+mycalsuffix,str(csource)+'.K1'+mycalsuffix, str(csource)+'.B1'+mycalsuffix]



Get flux scale


2022-02-11 08:51:52	SEVERE	fluxscale::Calibrater::fluxscale	Caught Exception:  Cannot find solutions for transfer field(s)
2022-02-11 08:51:52	SEVERE	fluxscale::::	Exception Reported: Error in Calibrater::fluxscale.
2022-02-11 08:51:52	SEVERE	fluxscale::::casa	Task fluxscale raised an exception of class RuntimeError with the following message: Error in Calibrater::fluxscale.


RuntimeError: Error in Calibrater::fluxscale.